In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime as dt
import torch
from tqdm import tqdm, tqdm_notebook,notebook
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max< np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv")
train = reduce_mem_usage(train)

In [ ]:
calendar = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
calendar = reduce_mem_usage(calendar)

In [ ]:
date_index = calendar['date']
dates = date_index[0:1913]
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in dates]
train['item_store_id'] = train.apply(lambda x: x['item_id']+'_'+x['store_id'],axis=1)
DF_Sales = train.loc[:,'d_1':'d_1913'].T
DF_Sales.columns = train['item_store_id'].values
DF_Sales = pd.DataFrame(DF_Sales).set_index([dates_list])
DF_Sales.head()

In [ ]:
def extract_info(column):
    
    x = {'CA':0, 'TX':0.5, 'WI':1}
    y = {'HOBBIES':0, 'HOUSEHOLD':0.5, 'FOODS':1}
    a = column.split('_')
    
    return x[a[-2]], y[a[0]]

In [ ]:
def training_data(batch_size=365,prediction_days = 1,strides=72):
    '''
    Each column represent a seprate store sales over the past 1913 days, will also try to include certain features later
    '''
    
    for count,column in enumerate(DF_Sales.columns):
        train = []
        Y = []
        
        loc, cat = extract_info(column)
        X= list(np.concatenate((DF_Sales[column],np.array([loc]*1913),np.array([cat]*1913))))
        X = np.array(X,dtype=np.float16)
        X = np.reshape(X,(1913,3))
    
        y = np.array(DF_Sales[column])
        
        
        for i in range(0,1913-batch_size-1-prediction_days,72):
            train.append(X[i:i+batch_size])
            Y.append(y[i+batch_size:i+batch_size+prediction_days])
            
        train = np.array(train)
        Y = np.array(Y)    
        yield train, Y

In [ ]:
def training_data(batch_size=365,prediction_days = 1,strides=72):
    '''
    Each column represent a seprate store sales over the past 1913 days, will also try to include certain features later
    '''
    
    for count,column in enumerate(DF_Sales.columns):
        train = []
        Y = []
        
        loc, cat = extract_info(column)
        X= list(np.concatenate((DF_Sales[column],np.array([loc]*1913),np.array([cat]*1913))))
        X = np.array(X,dtype=np.float16)
        X = np.reshape(X,(1913,3))
    
        y = np.array(DF_Sales[column])
        
        
        for i in range(0,1913-batch_size-1-prediction_days,72):
            train.append(X[i:i+batch_size])
            Y.append(y[i+batch_size:i+batch_size+prediction_days])
            
        train = np.array(train)
        Y = np.array(Y)    
        yield train, Y

In [ ]:
class CNN1d(nn.Module):
    def __init__(self ,n_filters, filter_sizes, output_dim, dropout):
        
        super(CNN1d,self).__init__()
       
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = 3, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, 516)
        self.fc1 = nn.Linear(516, 128)
        self.dropout = nn.Dropout(dropout) 
        
    def forward(self, embedded):
        
        
#         embedded = embedded.permute(0, 2, 1)
        
        
        #embedded = [batch size, emb dim, sent len]
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]  
        cat = self.fc(cat)
        return self.fc1(cat)

In [ ]:
class Linear_layer(nn.Module):
    def __init__(self):
        super(Linear_layer,self).__init__()
        
        self.linear = nn.Linear(128,30)
        self.linear1 = nn.Linear(30,1)
    def forward(self,x):
        
        x = self.linear(x)
        x = self.linear1(x)
        return x    

In [ ]:
def 

In [ ]:
# class Combine(nn.Module):
#     def __init__(self,n_filters, filter_sizes, output_dim, dropout=0.1):
#         super(Combine,self).__init__()
        
#         self.cnn = CNN1d(n_filters, filter_sizes, output_dim, dropout)
#         self.linear = Linear_layer(output_dim, 1)
    
#     def forward(self, x):
        
#         x = self.cnn(x)
#         x = self.linear(x)
#         return x

In [ ]:
cnn = CNN1d(64,[2,3,4,7,15, 30,60,90,120],128,dropout= 0.1)
linear = Linear_layer()

In [ ]:
# model = Combine(64,[2,3,4,7,15, 30,60,90,120],128,dropout= 0.1)
# model.cuda()
optim_cnn = torch.optim.Adam(cnn.parameters(), lr=0.0001)
optim_linear = torch.optim.Adam(linear.parameters(), lr =0.001)

Closs = nn.MSELoss()

In [ ]:
print(cnn)

In [ ]:
N_epoch =10
for epoch in notebook.tqdm(range(N_epoch),desc = "Epoch"):
    loss = 0
    for x,y in notebook.tqdm(training_data(128, 1, 72),desc ="Batch progress"):
        
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(y).float()
        x = x.permute(0,2,1)
        
        x = cnn(x)
        x = linear(x)
        loss_ = Closs(x,y)

        optim_cnn.zero_grad()
        optim_linear.zero_grad()
        
        loss_.backward()
        optim_linear.step()
        optim_cnn.step()
        loss += loss_.item()
        
    print("Epoch {} loss {:.2f}".format(epoch+1,loss))

In [ ]:
def lgb_dataset(sample_size = 30000):
    
    
    